In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/water-potability/water_potability.csv')
df

## **Exploratory Data Analysis**

In [ ]:
# checking missing value
df_missing = df.isna().sum()
print(df_missing[df_missing > 0])

In [ ]:
# potability check value
sns.countplot(data=df, x='Potability')

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(data=df.corr(), annot=True)
plt.show()

have a bad correlation

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = df.drop('Potability', axis=1)
y = df.Potability

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
numerical_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer(strategy='median'))
])

In [ ]:
preprocessor = ColumnTransformer([
    ('numeric', numerical_pipeline, X_train.columns)
])

In [ ]:
pipeline = Pipeline([
    ('pre', preprocessor),
    ('algo', RandomForestClassifier(random_state=42))
])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
pipeline.score(X_train, y_train), pipeline.score(X_test, y_test)

In [ ]:
pipeline.get_params()

In [ ]:
parameter = {
    'algo__n_estimators':[100,200, 350, 500], 
    'algo__min_samples_leaf':[2, 10, 30]
}

In [ ]:
model = GridSearchCV(pipeline, parameter, cv=5, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train), model.score(X_test, y_test)

## **Evalution**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f_acc = f1_score(y_test, y_pred)

print(accuracy)
print(f_acc)

In [ ]:
output = pd.DataFrame({'Potability': y_pred})
output.to_csv('submission.csv', index=False)